# Research: Robustness Analysis - BTC MACD-ADX Strategy

## Objectif de recherche

Valider la robustesse de la strategie BTC-MACD-ADX sur une periode etendue (2019-2025) et evaluer la valeur ajoutee de l'approche **adaptive ADX percentile** par rapport aux seuils fixes.

### Innovation cle: Seuils ADX adaptatifs

La strategie actuelle utilise des seuils ADX **dynamiques** calcules sur une fenetre glissante de 140 barres:
- **Seuil haut**: 86e percentile de l'ADX sur les 140 dernieres barres
- **Seuil bas**: 6e percentile de l'ADX sur les 140 dernieres barres

Cette approche s'adapte automatiquement aux regimes de volatilite changeants du marche, contrairement aux seuils fixes (ex: ADX > 25).

## Hypotheses

1. **H1**: La fenetre adaptative de 140 barres reste stable sur 2019-2025
2. **H2**: MACD fournit des signaux de sortie plus precoces que simple EMA cross durant les crashes
3. **H3**: ADX adaptatif surperforme ADX fixe (>25) en termes de Sharpe ratio
4. **H4**: Walk-forward validation confirme la robustesse des parametres
5. **H5**: La complexite MACD+ADX est justifiee vs strategie EMA simple

## Methodologie

1. Charger donnees BTCUSDT daily 2019-01-01 → 2026-02-17
2. Detecter regimes de marche (bull/bear/crash/recovery)
3. Calculer indicateurs MACD et ADX
4. Backtester avec parametres actuels sur chaque regime
5. Comparer: ADX adaptatif vs fixe, MACD vs EMA cross
6. Walk-forward validation sur parametres ADX window
7. Analyser la sensibilite des parametres

## Parametres actuels

- **MACD**: Fast=12, Slow=26, Signal=9 (standard)
- **ADX**: Period=25, Window=140, Lower Percentile=6, Upper Percentile=86
- **Periode actuelle**: 2021-04-09 → Now (Sharpe 1.224, CAGR 38.1%)
- **Periode cible**: 2019-01-01 → 2025-12-31
- **Sharpe attendu apres extension**: 0.8-1.0

---

## 1. Setup et chargement des donnees

Initialisation de QuantBook et chargement des donnees historiques BTCUSDT en resolution daily.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

# Load BTC-USD data from Yahoo Finance as proxy
# Note: Using BTC-USD instead of BTCUSDT due to data availability
print("Loading BTC-USD historical data from Yahoo Finance...")

ticker = yf.Ticker("BTC-USD")
df_daily = ticker.history(start="2019-01-01", end="2026-02-17", interval="1d")

# Standardize column names
df_daily.columns = [col.lower() for col in df_daily.columns]
df_daily = df_daily[['open', 'high', 'low', 'close', 'volume']]

# Clean data
df_daily = df_daily.dropna()

print(f"Data loaded: {len(df_daily)} daily bars from {df_daily.index[0]} to {df_daily.index[-1]}")
print(f"\nFirst 5 rows:")
print(df_daily.head())
print(f"\nLast 5 rows:")
print(df_daily.tail())

print(f"\nPrice range: ${df_daily['close'].min():.2f} - ${df_daily['close'].max():.2f}")
print(f"Current price: ${df_daily['close'].iloc[-1]:.2f}")

Loading BTC-USD historical data from Yahoo Finance...


Data loaded: 2604 daily bars from 2019-01-01 00:00:00+00:00 to 2026-02-16 00:00:00+00:00

First 5 rows:
                                  open         high          low        close  \
Date                                                                            
2019-01-01 00:00:00+00:00  3746.713379  3850.913818  3707.231201  3843.520020   
2019-01-02 00:00:00+00:00  3849.216309  3947.981201  3817.409424  3943.409424   
2019-01-03 00:00:00+00:00  3931.048584  3935.685059  3826.222900  3836.741211   
2019-01-04 00:00:00+00:00  3832.040039  3865.934570  3783.853760  3857.717529   
2019-01-05 00:00:00+00:00  3851.973877  3904.903076  3836.900146  3845.194580   

                               volume  
Date                                   
2019-01-01 00:00:00+00:00  4324200990  
2019-01-02 00:00:00+00:00  5244856836  
2019-01-03 00:00:00+00:00  4530215219  
2019-01-04 00:00:00+00:00  4847965467  
2019-01-05 00:00:00+00:00  5137609824  

Last 5 rows:
                                  

## 2. Detection des regimes de marche

Identification des phases bull/bear/crash/recovery pour analyser la performance de la strategie dans differents contextes.

In [2]:
# Define market regimes based on historical BTC events
regimes = [
    {'name': 'Bear 2019', 'start': '2019-01-01', 'end': '2019-06-30', 'type': 'bear'},
    {'name': 'Bull 2019-2020', 'start': '2019-07-01', 'end': '2020-03-11', 'type': 'bull'},
    {'name': 'COVID Crash', 'start': '2020-03-12', 'end': '2020-03-16', 'type': 'crash'},
    {'name': 'COVID Recovery', 'start': '2020-03-17', 'end': '2020-12-31', 'type': 'recovery'},
    {'name': 'Bull 2021', 'start': '2021-01-01', 'end': '2021-04-14', 'type': 'bull'},
    {'name': 'Correction 2021', 'start': '2021-04-15', 'end': '2021-07-20', 'type': 'bear'},
    {'name': 'Bull 2021 H2', 'start': '2021-07-21', 'end': '2021-11-09', 'type': 'bull'},
    {'name': 'Bear 2022', 'start': '2021-11-10', 'end': '2022-12-31', 'type': 'bear'},
    {'name': 'Sideways 2023', 'start': '2023-01-01', 'end': '2023-10-31', 'type': 'sideways'},
    {'name': 'Bull 2024-2025', 'start': '2023-11-01', 'end': '2025-12-31', 'type': 'bull'}
]

# Calculate regime statistics
regime_stats = []
for regime in regimes:
    mask = (df_daily.index >= regime['start']) & (df_daily.index <= regime['end'])
    regime_data = df_daily[mask]
    
    if len(regime_data) > 0:
        start_price = regime_data['close'].iloc[0]
        end_price = regime_data['close'].iloc[-1]
        total_return = (end_price / start_price - 1) * 100
        
        # Volatility (annualized)
        returns = regime_data['close'].pct_change().dropna()
        volatility = returns.std() * np.sqrt(365) * 100
        
        regime_stats.append({
            'Regime': regime['name'],
            'Type': regime['type'],
            'Days': len(regime_data),
            'Return (%)': round(total_return, 2),
            'Volatility (%)': round(volatility, 2),
            'Start Price': round(start_price, 2),
            'End Price': round(end_price, 2)
        })

regime_df = pd.DataFrame(regime_stats)
print("Market Regimes Analysis:")
print(regime_df.to_string(index=False))
print(f"\nTotal period return: {((df_daily['close'].iloc[-1] / df_daily['close'].iloc[0]) - 1) * 100:.2f}%")

Market Regimes Analysis:
         Regime     Type  Days  Return (%)  Volatility (%)  Start Price  End Price
      Bear 2019     bear   181      181.44           69.57      3843.52   10817.16
 Bull 2019-2020     bull   255      -25.24           62.97     10583.13    7911.43
    COVID Crash    crash     5        0.88          173.20      4970.79    5014.48
 COVID Recovery recovery   290      454.99           60.96      5225.63   29001.72
      Bull 2021     bull   104      114.85           88.26     29374.15   63109.70
Correction 2021     bear    97      -52.92           91.54     63314.01   29807.35
   Bull 2021 H2     bull   112      108.57           66.73     32110.69   66971.83
      Bear 2022     bear   417      -74.54           63.15     64995.23   16547.50
  Sideways 2023 sideways   304      108.53           44.00     16625.08   34667.78
 Bull 2024-2025     bull   792      146.94           47.76     35437.25   87508.83

Total period return: 1691.15%


## 3. Calcul des indicateurs MACD et ADX

Implementation des indicateurs avec les parametres actuels:
- MACD (12, 26, 9)
- ADX (period=25) avec seuils adaptatifs (window=140, percentiles 6/86)

In [3]:
def compute_macd(close, fast=12, slow=26, signal=9):
    """Compute MACD indicator."""
    ema_fast = close.ewm(span=fast, adjust=False).mean()
    ema_slow = close.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

def compute_adx(high, low, close, period=25):
    """Compute ADX indicator."""
    # Directional movement
    plus_dm = high.diff()
    minus_dm = -low.diff()
    
    # Set negative values to 0
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    
    # Keep only the larger movement
    plus_dm = plus_dm.where(plus_dm > minus_dm, 0)
    minus_dm = minus_dm.where(minus_dm > plus_dm, 0)
    
    # True Range
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # Smooth TR and DM
    atr = tr.rolling(period).mean()
    plus_di = 100 * (plus_dm.rolling(period).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(period).mean() / atr)
    
    # DX and ADX
    dx = 100 * ((plus_di - minus_di).abs() / (plus_di + minus_di))
    adx = dx.rolling(period).mean()
    
    return adx

def adaptive_adx_thresholds(adx, window=140, lower_pct=6, upper_pct=86):
    """Compute rolling percentile-based ADX thresholds."""
    lower_threshold = adx.rolling(window).quantile(lower_pct / 100)
    upper_threshold = adx.rolling(window).quantile(upper_pct / 100)
    return lower_threshold, upper_threshold

# Calculate indicators
macd, signal, histogram = compute_macd(df_daily['close'])
adx = compute_adx(df_daily['high'], df_daily['low'], df_daily['close'])
adx_lower, adx_upper = adaptive_adx_thresholds(adx)

# Add to dataframe
df_daily['macd'] = macd
df_daily['signal'] = signal
df_daily['histogram'] = histogram
df_daily['adx'] = adx
df_daily['adx_lower'] = adx_lower
df_daily['adx_upper'] = adx_upper

# Drop NaN values
df_daily = df_daily.dropna()

print(f"Indicators calculated: {len(df_daily)} valid bars")
print(f"\nSample data with indicators:")
print(df_daily[['close', 'macd', 'signal', 'adx', 'adx_lower', 'adx_upper']].tail(10))

Indicators calculated: 2417 valid bars

Sample data with indicators:
                                  close         macd       signal        adx  \
Date                                                                           
2026-02-07 00:00:00+00:00  69281.968750 -5652.638254 -3853.692507  38.669991   
2026-02-08 00:00:00+00:00  70264.726562 -5695.974251 -4222.148856  39.353025   
2026-02-09 00:00:00+00:00  70120.781250 -5676.498385 -4513.018762  40.072743   
2026-02-10 00:00:00+00:00  68793.960938 -5702.393291 -4750.893668  41.127331   
2026-02-11 00:00:00+00:00  66991.968750 -5801.445449 -4961.004024  41.812231   
2026-02-12 00:00:00+00:00  66221.843750 -5874.371550 -5143.677529  42.861895   
2026-02-13 00:00:00+00:00  68857.843750 -5654.283777 -5245.798779  44.171912   
2026-02-14 00:00:00+00:00  69767.625000 -5344.838911 -5265.606805  46.248709   
2026-02-15 00:00:00+00:00  68788.187500 -5119.618274 -5236.409099  48.433316   
2026-02-16 00:00:00+00:00  68843.156250 -4880.43515

## 4. Backtest vectorise: Strategie actuelle vs Baselines

Comparaison de 4 strategies:
1. **MACD + ADX Adaptatif** (strategie actuelle)
2. **MACD + ADX Fixe** (ADX > 25 pour entree)
3. **MACD seul** (sans filtre ADX)
4. **EMA Cross simple** (12/26, baseline)

Metriques: Sharpe ratio, CAGR, Max Drawdown, Win Rate

In [4]:
def backtest_strategy(df, entry_func, exit_func, name):
    """Vectorized backtest engine."""
    position = pd.Series(0, index=df.index)
    in_position = False
    
    for i in range(1, len(df)):
        if not in_position:
            if entry_func(df.iloc[:i+1]):
                in_position = True
        else:
            if exit_func(df.iloc[:i+1]):
                in_position = False
        
        position.iloc[i] = 1 if in_position else 0
    
    # Calculate returns
    position = position.shift(1).fillna(0)
    returns = df['close'].pct_change() * position
    
    # Metrics
    total_return = (1 + returns).cumprod().iloc[-1] - 1
    days = len(df)
    years = days / 365
    cagr = ((1 + total_return) ** (1 / years) - 1) if years > 0 else 0
    
    sharpe = returns.mean() / returns.std() * np.sqrt(365) if returns.std() > 0 else 0
    
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min()
    
    trades = position.diff().abs().sum() / 2
    win_rate = (returns[returns > 0].count() / returns[returns != 0].count()) if returns[returns != 0].count() > 0 else 0
    
    return {
        'Strategy': name,
        'Sharpe': round(sharpe, 3),
        'CAGR (%)': round(cagr * 100, 2),
        'Total Return (%)': round(total_return * 100, 2),
        'Max DD (%)': round(max_dd * 100, 2),
        'Win Rate (%)': round(win_rate * 100, 2),
        'Trades': int(trades),
        'Days': days
    }

# Strategy 1: MACD + ADX Adaptive (current)
def entry_adaptive(df):
    last = df.iloc[-1]
    return last['histogram'] > 0 and last['adx'] > last['adx_upper']

def exit_adaptive(df):
    last = df.iloc[-1]
    return last['histogram'] < 0 or last['adx'] < last['adx_lower']

# Strategy 2: MACD + ADX Fixed
def entry_fixed(df):
    last = df.iloc[-1]
    return last['histogram'] > 0 and last['adx'] > 25

def exit_fixed(df):
    last = df.iloc[-1]
    return last['histogram'] < 0 or last['adx'] < 15

# Strategy 3: MACD only
def entry_macd(df):
    return df.iloc[-1]['histogram'] > 0

def exit_macd(df):
    return df.iloc[-1]['histogram'] < 0

# Strategy 4: EMA Cross
df_daily['ema_12'] = df_daily['close'].ewm(span=12, adjust=False).mean()
df_daily['ema_26'] = df_daily['close'].ewm(span=26, adjust=False).mean()

def entry_ema(df):
    return df.iloc[-1]['ema_12'] > df.iloc[-1]['ema_26']

def exit_ema(df):
    return df.iloc[-1]['ema_12'] < df.iloc[-1]['ema_26']

# Run backtests
results = []
results.append(backtest_strategy(df_daily, entry_adaptive, exit_adaptive, 'MACD+ADX Adaptive'))
results.append(backtest_strategy(df_daily, entry_fixed, exit_fixed, 'MACD+ADX Fixed'))
results.append(backtest_strategy(df_daily, entry_macd, exit_macd, 'MACD Only'))
results.append(backtest_strategy(df_daily, entry_ema, exit_ema, 'EMA Cross'))

# Buy & Hold baseline
bh_return = (df_daily['close'].iloc[-1] / df_daily['close'].iloc[0]) - 1
bh_years = len(df_daily) / 365
bh_cagr = ((1 + bh_return) ** (1 / bh_years) - 1)
results.append({
    'Strategy': 'Buy & Hold',
    'Sharpe': 0,
    'CAGR (%)': round(bh_cagr * 100, 2),
    'Total Return (%)': round(bh_return * 100, 2),
    'Max DD (%)': 0,
    'Win Rate (%)': 0,
    'Trades': 1,
    'Days': len(df_daily)
})

results_df = pd.DataFrame(results)
print("\nStrategy Comparison Results:")
print(results_df.to_string(index=False))


Strategy Comparison Results:
         Strategy  Sharpe  CAGR (%)  Total Return (%)  Max DD (%)  Win Rate (%)  Trades  Days
MACD+ADX Adaptive  -0.035     -2.90            -17.68      -42.19         44.33      21  2417
   MACD+ADX Fixed   0.491     11.45            104.97      -56.91         47.68      61  2417
        MACD Only   0.939     34.23            602.49      -48.71         49.47      83  2417
        EMA Cross   1.019     40.28            840.48      -57.39         51.27      35  2417
       Buy & Hold   0.000     31.11            501.21        0.00          0.00       1  2417


### Interpretation des resultats

**Points cles a analyser:**

1. **Sharpe Ratio**: La strategie ADX adaptative doit montrer un Sharpe superieur aux autres approches
2. **CAGR vs Max DD**: Trade-off risque/rendement - ADX adaptatif devrait optimiser ce ratio
3. **Win Rate**: Un taux de reussite plus eleve indique une meilleure selection des trades
4. **Nombre de trades**: ADX adaptatif devrait filtrer efficacement (moins de trades que MACD seul)

**Hypothese H3 validee si**: Sharpe(ADX Adaptive) > Sharpe(ADX Fixed) > Sharpe(MACD Only)

## 5. Analyse de sensibilite: ADX Window et Percentiles

Test systematique des parametres ADX:
- Windows: [80, 100, 140, 180, 200]
- Percentiles: [(5, 85), (6, 86), (10, 90)]

Objectif: Confirmer que window=140, percentiles=(6, 86) sont optimaux.

In [5]:
# Grid search on ADX parameters
windows = [80, 100, 140, 180, 200]
percentiles = [(5, 85), (6, 86), (10, 90)]

sensitivity_results = []

for window in windows:
    for lower_pct, upper_pct in percentiles:
        # Recalculate adaptive thresholds
        adx_lower_temp = adx.rolling(window).quantile(lower_pct / 100)
        adx_upper_temp = adx.rolling(window).quantile(upper_pct / 100)
        
        df_temp = df_daily.copy()
        df_temp['adx_lower'] = adx_lower_temp
        df_temp['adx_upper'] = adx_upper_temp
        df_temp = df_temp.dropna()
        
        # Backtest
        result = backtest_strategy(
            df_temp, 
            entry_adaptive, 
            exit_adaptive, 
            f'W{window}_P{lower_pct}-{upper_pct}'
        )
        result['Window'] = window
        result['Lower_Pct'] = lower_pct
        result['Upper_Pct'] = upper_pct
        sensitivity_results.append(result)

sensitivity_df = pd.DataFrame(sensitivity_results)
sensitivity_df = sensitivity_df.sort_values('Sharpe', ascending=False)

print("\nADX Parameter Sensitivity Analysis (Top 10):")
print(sensitivity_df[['Window', 'Lower_Pct', 'Upper_Pct', 'Sharpe', 'CAGR (%)', 'Max DD (%)', 'Trades']].head(10).to_string(index=False))

print(f"\nCurrent params rank: {sensitivity_df[sensitivity_df['Strategy'] == 'W140_P6-86'].index[0] + 1} / {len(sensitivity_df)}")


ADX Parameter Sensitivity Analysis (Top 10):
 Window  Lower_Pct  Upper_Pct  Sharpe  CAGR (%)  Max DD (%)  Trades
     80          5         85   0.358      5.71      -39.19      21
     80          6         86   0.356      5.67      -39.19      21
     80         10         90   0.333      5.07      -39.74      19
    180          5         85   0.327      5.00      -38.03      22
    180          6         86   0.267      3.54      -38.83      20
    100          5         85   0.240      2.87      -37.12      19
    100          6         86   0.219      2.44      -37.12      18
    100         10         90   0.176      1.57      -35.27      15
    200          5         85   0.163      1.19      -41.08      21
    140         10         90   0.077     -0.40      -37.12      16

Current params rank: 8 / 15


### Interpretation: Stabilite des parametres

**Hypothese H1 validee si:**
- Les parametres actuels (140, 6, 86) figurent dans le top 3 des configurations
- Les variations de Sharpe entre configurations proches sont < 10%
- Pas de configuration dominante extreme (signe d'overfitting)

**Si H1 refutee:** Recommander d'ajuster window ou percentiles selon les resultats.

## 6. Walk-Forward Validation

Validation temporelle pour eviter le look-ahead bias:
- **Train period**: 252 jours (1 an)
- **Test period**: 63 jours (3 mois)
- **Rolling window**: Deplacement de 63 jours

Calcul du **Walk-Forward Efficiency (WFE)**: ratio performance out-of-sample / in-sample.

**WFE > 60%**: Parametres robustes, pas d'overfitting

In [6]:
# Walk-forward validation
train_days = 252  # 1 year
test_days = 63    # 3 months
step_days = 63    # Rolling window step

wf_results = []
start_idx = train_days

while start_idx + test_days <= len(df_daily):
    # Train period
    train_data = df_daily.iloc[start_idx - train_days:start_idx]
    
    # Test period
    test_data = df_daily.iloc[start_idx:start_idx + test_days]
    
    # Optimize on train (use grid search on train data)
    best_sharpe_train = -999
    best_params = None
    
    for window in [100, 140, 180]:
        for lower_pct, upper_pct in [(6, 86), (10, 90)]:
            adx_lower_temp = adx.rolling(window).quantile(lower_pct / 100)
            adx_upper_temp = adx.rolling(window).quantile(upper_pct / 100)
            
            df_temp = train_data.copy()
            df_temp['adx_lower'] = adx_lower_temp.loc[df_temp.index]
            df_temp['adx_upper'] = adx_upper_temp.loc[df_temp.index]
            df_temp = df_temp.dropna()
            
            if len(df_temp) < 50:
                continue
            
            result = backtest_strategy(df_temp, entry_adaptive, exit_adaptive, 'train')
            if result['Sharpe'] > best_sharpe_train:
                best_sharpe_train = result['Sharpe']
                best_params = (window, lower_pct, upper_pct)
    
    # Test with best params
    if best_params:
        window, lower_pct, upper_pct = best_params
        adx_lower_temp = adx.rolling(window).quantile(lower_pct / 100)
        adx_upper_temp = adx.rolling(window).quantile(upper_pct / 100)
        
        df_temp = test_data.copy()
        df_temp['adx_lower'] = adx_lower_temp.loc[df_temp.index]
        df_temp['adx_upper'] = adx_upper_temp.loc[df_temp.index]
        df_temp = df_temp.dropna()
        
        if len(df_temp) >= 20:
            result_test = backtest_strategy(df_temp, entry_adaptive, exit_adaptive, 'test')
            
            wf_results.append({
                'Train_Start': train_data.index[0].strftime('%Y-%m-%d'),
                'Test_Start': test_data.index[0].strftime('%Y-%m-%d'),
                'Train_Sharpe': round(best_sharpe_train, 3),
                'Test_Sharpe': round(result_test['Sharpe'], 3),
                'Window': window,
                'Percentiles': f"{lower_pct}-{upper_pct}"
            })
    
    start_idx += step_days

wf_df = pd.DataFrame(wf_results)

if len(wf_df) > 0:
    # Calculate WFE (Walk-Forward Efficiency)
    avg_train_sharpe = wf_df['Train_Sharpe'].mean()
    avg_test_sharpe = wf_df['Test_Sharpe'].mean()
    wfe = (avg_test_sharpe / avg_train_sharpe * 100) if avg_train_sharpe > 0 else 0
    
    print("\nWalk-Forward Validation Results:")
    print(wf_df.to_string(index=False))
    print(f"\nAverage Train Sharpe: {avg_train_sharpe:.3f}")
    print(f"Average Test Sharpe: {avg_test_sharpe:.3f}")
    print(f"Walk-Forward Efficiency: {wfe:.2f}%")
    print(f"\nValidation: {'PASS - Robust parameters' if wfe > 60 else 'FAIL - Potential overfitting'}")
else:
    print("Insufficient data for walk-forward validation")


Walk-Forward Validation Results:
Train_Start Test_Start  Train_Sharpe  Test_Sharpe  Window Percentiles
 2019-07-07 2020-03-15        -1.054        2.416     100        6-86
 2019-09-08 2020-05-17         0.919        0.000     180        6-86
 2019-11-10 2020-07-19         1.423       -1.402     180        6-86
 2020-01-12 2020-09-20         1.768        4.925     180        6-86
 2020-03-15 2020-11-22         2.784        0.000     180        6-86
 2020-05-17 2021-01-24         1.318        0.000     180        6-86
 2020-07-19 2021-03-28         1.318        0.000     180        6-86
 2020-09-20 2021-05-30         1.542       -1.359     180        6-86
 2020-11-22 2021-08-01        -0.678        0.000     100        6-86
 2021-01-24 2021-10-03        -0.678       -3.243     100        6-86
 2021-03-28 2021-12-05        -0.678       -3.037     180       10-90
 2021-05-30 2022-02-06        -0.884        0.000     100        6-86
 2021-08-01 2022-04-10        -1.504       -0.322     18

### Interpretation: Robustesse temporelle

**Hypothese H4 validee si:**
- WFE > 60% (performance out-of-sample >= 60% de in-sample)
- Sharpe test positif dans > 70% des periodes
- Pas de degradation systematique sur periodes recentes

**Action recommandee si WFE > 60%:**
- Etendre SetStartDate(2019, 1, 1) dans Main.cs
- Conserver parametres actuels (140, 6, 86)

## 7. Conclusions et Recommandations

Synthese des findings et proposition d'ameliorations pour la strategie.

In [7]:
# Summary of findings
print("="*80)
print("RESEARCH FINDINGS SUMMARY")
print("="*80)

print("\n1. STRATEGY COMPARISON (2019-2025):")
print(results_df[['Strategy', 'Sharpe', 'CAGR (%)', 'Max DD (%)', 'Win Rate (%)']].to_string(index=False))

adaptive_sharpe = results_df[results_df['Strategy'] == 'MACD+ADX Adaptive']['Sharpe'].values[0]
fixed_sharpe = results_df[results_df['Strategy'] == 'MACD+ADX Fixed']['Sharpe'].values[0]
macd_sharpe = results_df[results_df['Strategy'] == 'MACD Only']['Sharpe'].values[0]
ema_sharpe = results_df[results_df['Strategy'] == 'EMA Cross']['Sharpe'].values[0]

print(f"\n   H3 - ADX Adaptive > Fixed: {'CONFIRMED' if adaptive_sharpe > fixed_sharpe else 'REJECTED'}")
print(f"   Adaptive outperformance: {((adaptive_sharpe / fixed_sharpe - 1) * 100):.2f}%")

print(f"\n   H5 - MACD+ADX > EMA Cross: {'CONFIRMED' if adaptive_sharpe > ema_sharpe else 'REJECTED'}")
print(f"   Complexity justified: {((adaptive_sharpe / ema_sharpe - 1) * 100):.2f}% improvement")

print("\n2. PARAMETER SENSITIVITY:")
best_config = sensitivity_df.iloc[0]
current_config = sensitivity_df[sensitivity_df['Strategy'] == 'W140_P6-86'].iloc[0]
print(f"   Best config: Window={best_config['Window']}, Percentiles={best_config['Lower_Pct']}-{best_config['Upper_Pct']}, Sharpe={best_config['Sharpe']}")
print(f"   Current config: Window=140, Percentiles=6-86, Sharpe={current_config['Sharpe']}")
print(f"   Current rank: {sensitivity_df[sensitivity_df['Strategy'] == 'W140_P6-86'].index[0] + 1} / {len(sensitivity_df)}")

print(f"\n   H1 - Parameter Stability: {'CONFIRMED' if sensitivity_df[sensitivity_df['Strategy'] == 'W140_P6-86'].index[0] < 3 else 'PARTIAL'}")

if len(wf_df) > 0:
    print("\n3. WALK-FORWARD VALIDATION:")
    print(f"   Average Test Sharpe: {avg_test_sharpe:.3f}")
    print(f"   Walk-Forward Efficiency: {wfe:.2f}%")
    print(f"   H4 - Robustness: {'CONFIRMED (WFE > 60%)' if wfe > 60 else 'REJECTED (WFE < 60%)'}")

print("\n" + "="*80)
print("RECOMMENDATIONS")
print("="*80)

recommendations = []

if adaptive_sharpe > fixed_sharpe:
    recommendations.append("[KEEP] Adaptive ADX approach provides superior risk-adjusted returns")
else:
    recommendations.append("[REVIEW] Consider reverting to fixed ADX thresholds")

if len(wf_df) > 0 and wfe > 60:
    recommendations.append(f"[ACTION] Extend backtest period to 2019-01-01 in Main.cs (expected Sharpe: {avg_test_sharpe:.2f})")
    recommendations.append("[KEEP] Current parameters (140, 6, 86) are robust across time")
else:
    recommendations.append("[CAUTION] Parameters may be overfitted, consider re-optimization")

if sensitivity_df.iloc[0]['Strategy'] != 'W140_P6-86':
    best = sensitivity_df.iloc[0]
    recommendations.append(f"[OPTIMIZE] Consider updating to Window={best['Window']}, Percentiles={best['Lower_Pct']}-{best['Upper_Pct']} (Sharpe gain: {(best['Sharpe'] - current_config['Sharpe']):.3f})")

if adaptive_sharpe > macd_sharpe * 1.1:
    recommendations.append("[KEEP] ADX filter adds significant value (>10% Sharpe improvement)")
else:
    recommendations.append("[SIMPLIFY] Consider removing ADX filter for simplicity")

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print("\n" + "="*80)

RESEARCH FINDINGS SUMMARY

1. STRATEGY COMPARISON (2019-2025):
         Strategy  Sharpe  CAGR (%)  Max DD (%)  Win Rate (%)
MACD+ADX Adaptive  -0.035     -2.90      -42.19         44.33
   MACD+ADX Fixed   0.491     11.45      -56.91         47.68
        MACD Only   0.939     34.23      -48.71         49.47
        EMA Cross   1.019     40.28      -57.39         51.27
       Buy & Hold   0.000     31.11        0.00          0.00

   H3 - ADX Adaptive > Fixed: REJECTED
   Adaptive outperformance: -107.13%

   H5 - MACD+ADX > EMA Cross: REJECTED
   Complexity justified: -103.43% improvement

2. PARAMETER SENSITIVITY:
   Best config: Window=80, Percentiles=5-85, Sharpe=0.358
   Current config: Window=140, Percentiles=6-86, Sharpe=-0.035
   Current rank: 8 / 15

   H1 - Parameter Stability: PARTIAL

3. WALK-FORWARD VALIDATION:
   Average Test Sharpe: 0.168
   Walk-Forward Efficiency: 26.09%
   H4 - Robustness: REJECTED (WFE < 60%)

RECOMMENDATIONS
1. [REVIEW] Consider reverting to fixed 